# Clustering - K-Means | פילוח לקוחות כרטיסי אשראי

## רקע עסקי

חברת **"CardWise"** - חברת כרטיסי אשראי מובילה בישראל. עם הגדילה בבסיס הלקוחות, החברה צריכה להבין את התנהגות הלקוחות כדי:
- לייעל שיווק ממוקד
- למזער סיכוני אשראי
- להציע מבצעים מותאמים אישית

**מטרה:** לזהות קבוצות לקוחות דומות (segments) באמצעות K-Means Clustering.

## מה זה K-Means? (הסבר תיאורטי)

K-Means הוא אלגוריתם **Unsupervised Learning** (למידה לא מפוקחת) - אין לנו "תשובה נכונה" (label) לכל דוגמה. האלגוריתם מנסה **לגלות מבנה** בנתונים בעצמו.

### שלבי האלגוריתם:
1. בחר k נקודות מרכז (centroids) אקראיות
2. שייך כל נקודת נתונים למרכז הקרוב אליה (לפי מרחק אוקלידי)
3. חשב מחדש את המרכזים (ממוצע כל הנקודות בקלאסטר)
4. חזור על 2-3 עד שהמרכזים לא משתנים (התכנסות)

**מרחק אוקלידי:** `d(A, B) = sqrt( (x1-x2)² + (y1-y2)² + ... )`

---
## הכנות סביבה וייבואים

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# הגדרת סגנון כללי לגרפים
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

---
## חלק א' - טעינת הנתונים ובדיקה ראשונית

### למה השלב הזה חשוב?
לפני כל מודל, **חייבים** להכיר את הנתונים: מבנה, סוגי משתנים, ערכים חסרים. טעויות בשלב הזה יגרמו לתוצאות שגויות בהמשך.

In [ ]:
# שלב 1: טעינת הנתונים
df_customers = pd.read_csv("Customer Data.csv")

print(f"מספר שורות (לקוחות): {df_customers.shape[0]}")
print(f"מספר עמודות (תכונות): {df_customers.shape[1]}")
df_customers.head()

In [ ]:
# סוגי עמודות ומידע כללי
df_customers.info()

In [ ]:
# סטטיסטיקות תיאוריות
df_customers.describe()

### בדיקת ערכים חסרים וטיפול

יש 3 שיטות לטפל בערכים חסרים:

| שיטה | יתרון | חיסרון |
|------|-------|--------|
| **מחיקת שורות** | פשוט | מאבדים מידע (313 לקוחות!) |
| **Mean (ממוצע)** | קל לחישוב | **רגיש ל-outliers** |
| **Median (חציון)** | **עמיד ל-outliers** | קצת פחות "מדויק" |

בנתוני כרטיסי אשראי יש הרבה outliers (לקוחות עם הוצאות חריגות), לכן **Median הוא הבחירה הנכונה**.

In [ ]:
# בדיקת ערכים חסרים
missing = df_customers.isnull().sum()
missing_cols = missing[missing > 0]
print("ערכים חסרים:")
print(missing_cols)

# טיפול: מילוי ב-median
print("\nמלאים ב-median:")
for col in missing_cols.index:
    median_val = df_customers[col].median()
    df_customers[col] = df_customers[col].fillna(median_val)
    print(f"  {col}: מולא ב-median = {median_val:,.2f}")

print(f"\nאימות: ערכים חסרים לאחר טיפול: {df_customers.isnull().sum().sum()}")

---
## חלק ב' - ניתוח חקר נתונים (EDA)

### למה EDA?
EDA (**Exploratory Data Analysis**) עוזר לנו להבין את הנתונים **לפני** שמריצים מודלים - לזהות דפוסים, קשרים בין משתנים, ונקודות חריגות.

### 1. מטריצת קורלציה

קורלציה מודדת את עוצמת הקשר הליניארי בין שני משתנים (בין -1 ל-1):
- קרוב ל-**1**: קשר חיובי חזק (כשאחד עולה, גם השני)
- קרוב ל-**(-1)**: קשר שלילי חזק
- קרוב ל-**0**: אין קשר ליניארי

In [ ]:
# מטריצת קורלציה בין תכונות עיקריות
key_features = ["BALANCE", "PURCHASES", "CASH_ADVANCE", "CREDIT_LIMIT", "PAYMENTS"]
corr_customers = df_customers[key_features].corr()
print(corr_customers.round(3))

**ממצאים עיקריים:**
- **PURCHASES ↔ PAYMENTS**: קורלציה חיובית – לקוחות שקונים יותר גם משלמים יותר
- **BALANCE ↔ CASH_ADVANCE**: קורלציה חיובית – משיכות מזומן מעלות את היתרה
- **PURCHASES ↔ CASH_ADVANCE**: קורלציה נמוכה – לקוחות נוטים **לאחד מהשניים**, לא לשניהם

### 2. ויזואליזציות

In [ ]:
fig_eda, axes_eda = plt.subplots(2, 3, figsize=(20, 12))
fig_eda.suptitle("EDA - Customer Credit Card Data", fontsize=18, fontweight="bold", y=1.02)

# גרף 1: היסטוגרמה של BALANCE
# הסבר: היסטוגרמה מראה את התפלגות הערכים.
# אם מוטה ימינה (right-skewed) = הרבה לקוחות עם ערכים נמוכים,
# ומעט לקוחות עם ערכים גבוהים מאוד (outliers).
axes_eda[0, 0].hist(df_customers["BALANCE"], bins=50, color="steelblue", edgecolor="white", alpha=0.8)
axes_eda[0, 0].set_title("Distribution of BALANCE", fontsize=13, fontweight="bold")
axes_eda[0, 0].set_xlabel("Balance")
axes_eda[0, 0].set_ylabel("Count")

# גרף 2: היסטוגרמה של PURCHASES
axes_eda[0, 1].hist(df_customers["PURCHASES"], bins=50, color="coral", edgecolor="white", alpha=0.8)
axes_eda[0, 1].set_title("Distribution of PURCHASES", fontsize=13, fontweight="bold")
axes_eda[0, 1].set_xlabel("Purchases")
axes_eda[0, 1].set_ylabel("Count")

# גרף 3: Scatter plot – PURCHASES vs CASH_ADVANCE
# הסבר: אם הנקודות מתפזרות לאורך הצירים (ולא באלכסון),
# זה מצביע על שני "סגנונות" שונים של לקוחות.
axes_eda[0, 2].scatter(df_customers["PURCHASES"], df_customers["CASH_ADVANCE"],
                       alpha=0.3, color="purple", edgecolor="white", s=15)
axes_eda[0, 2].set_title("Purchases vs Cash Advance", fontsize=13, fontweight="bold")
axes_eda[0, 2].set_xlabel("Purchases")
axes_eda[0, 2].set_ylabel("Cash Advance")

# גרף 4: מטריצת קורלציה (Heatmap)
sns.heatmap(corr_customers, annot=True, fmt=".2f", cmap="RdBu_r", center=0,
            ax=axes_eda[1, 0], cbar_kws={"shrink": 0.8}, linewidths=0.5, square=True)
axes_eda[1, 0].set_title("Correlation Matrix", fontsize=13, fontweight="bold")

# גרף 5: Boxplot של CASH_ADVANCE
# הסבר: הקו באמצע = חציון, הקופסה = Q1 עד Q3,
# השפמים = טווח סביר, נקודות מעבר = outliers
axes_eda[1, 1].boxplot(df_customers["CASH_ADVANCE"].dropna(), vert=True, patch_artist=True,
                       boxprops=dict(facecolor="lightyellow", color="orange"),
                       medianprops=dict(color="red", linewidth=2))
axes_eda[1, 1].set_title("Boxplot: CASH_ADVANCE", fontsize=13, fontweight="bold")
axes_eda[1, 1].set_ylabel("Cash Advance Amount")

# גרף 6: Boxplot של CREDIT_LIMIT
axes_eda[1, 2].boxplot(df_customers["CREDIT_LIMIT"].dropna(), vert=True, patch_artist=True,
                       boxprops=dict(facecolor="lightcyan", color="steelblue"),
                       medianprops=dict(color="red", linewidth=2))
axes_eda[1, 2].set_title("Boxplot: CREDIT_LIMIT", fontsize=13, fontweight="bold")
axes_eda[1, 2].set_ylabel("Credit Limit")

plt.tight_layout()
plt.savefig("clustering_eda_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()

### 3. זיהוי Outliers בעזרת שיטת IQR

In [ ]:
# זיהוי נקודות קיצון (Outliers) בשיטת IQR
for col_name in ["CASH_ADVANCE", "CREDIT_LIMIT"]:
    Q1 = df_customers[col_name].quantile(0.25)
    Q3 = df_customers[col_name].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    n_outliers = ((df_customers[col_name] < lower) | (df_customers[col_name] > upper)).sum()
    print(f"{col_name}: Q1={Q1:,.0f}, Q3={Q3:,.0f}, IQR={IQR:,.0f}")
    print(f"  גבול עליון: {upper:,.0f}, מספר outliers: {n_outliers}\n")

print("החלטה: לא מסירים את ה-outliers כי ב-Clustering")
print("הם עשויים לייצג סגמנט לגיטימי (למשל, לקוחות VIP).")

---
## חלק ג' - קדם-עיבוד והתאמה למודל

### למה Scaling הוא קריטי ב-K-Means?

K-Means מחשב **מרחקים** בין נקודות. בלי scaling:

| תכונה | טווח מקורי |
|--------|-------------|
| `BALANCE` | 0 - 19,043 |
| `PURCHASES_FREQUENCY` | 0 - 1 |

> **הבעיה:** BALANCE עם טווח של אלפים **ישלוט לחלוטין** על חישוב המרחק, ו-PURCHASES_FREQUENCY (0-1) יהיה חסר משמעות.
>
> **StandardScaler** מבצע: `X_scaled = (X - mean) / std` → כל תכונה מקבלת ממוצע=0, סטיית תקן=1.

In [ ]:
# בחירת תכונות רלוונטיות
# הסרנו:
#   CUST_ID – מזהה ייחודי, לא מידע מספרי רלוונטי
#   TENURE – כמעט אחיד אצל כל הלקוחות (רובם 12 חודשים), לא תורם לפילוח
cols_to_drop = ["CUST_ID", "TENURE"]
df_clustering = df_customers.drop(columns=cols_to_drop)

print(f"תכונות שנבחרו למודל ({df_clustering.shape[1]}):")
for i, col in enumerate(df_clustering.columns, 1):
    print(f"  {i}. {col}")

print(f"\nתכונות שהוסרו: {cols_to_drop}")

In [ ]:
# Standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clustering)

print("בוצע StandardScaler:")
print(f"  לפני – טווח BALANCE: [{df_clustering['BALANCE'].min():,.0f}, {df_clustering['BALANCE'].max():,.0f}]")
print(f"  לפני – טווח PURCHASES_FREQUENCY: [{df_clustering['PURCHASES_FREQUENCY'].min():.2f}, {df_clustering['PURCHASES_FREQUENCY'].max():.2f}]")
print(f"  אחרי – כל התכונות: ממוצע ≈ 0, סטיית תקן ≈ 1")

---
## חלק ד' - מציאת מספר הקלאסטרים (Elbow Method)

### מה זה Elbow Method?

מריצים K-Means עם k שונים ומודדים את ה-**Inertia** (WCSS):

`Inertia = סכום ריבועי המרחקים של כל נקודה מהמרכז של הקלאסטר שלה`

- ככל ש-k גדל → Inertia יורד
- מחפשים את ה-**"מרפק"** – הנקודה שבה קצב הירידה מואט משמעותית

In [ ]:
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(X_scaled)
    inertias.append(kmeans_temp.inertia_)
    print(f"  k={k}: Inertia = {kmeans_temp.inertia_:,.0f}")

In [ ]:
# גרף Elbow
optimal_k = 4

fig_elbow, ax_elbow = plt.subplots(figsize=(10, 6))
ax_elbow.plot(list(K_range), inertias, "bo-", linewidth=2, markersize=8)
ax_elbow.set_title("Elbow Method \u2013 Optimal Number of Clusters", fontsize=15, fontweight="bold")
ax_elbow.set_xlabel("Number of Clusters (k)", fontsize=12)
ax_elbow.set_ylabel("Inertia (Within-Cluster Sum of Squares)", fontsize=12)
ax_elbow.set_xticks(list(K_range))
ax_elbow.grid(True, alpha=0.3)
ax_elbow.axvline(x=optimal_k, color="red", linestyle="--", linewidth=2, label=f"Chosen k={optimal_k}")
ax_elbow.legend(fontsize=12)
plt.tight_layout()
plt.savefig("clustering_elbow.png", dpi=150, bbox_inches="tight")
plt.show()

### בחירה: k = 4

בגרף ה-Elbow רואים שהירידה ב-Inertia **מואטת משמעותית** אחרי k=4:
- מ-2 ל-3: ירידה של ~15,700
- מ-3 ל-4: ירידה של ~13,000
- מ-4 ל-5: ירידה של ~7,500 (התמתנה!)

k=4 הוא האיזון הטוב בין פשטות לאיכות פילוח.

---
## חלק ה' - אימון KMeans ופרופילינג של קלאסטרים

### פרמטרים:
- `n_clusters=4` – מספר הקלאסטרים שבחרנו
- `random_state=42` – מבטיח תוצאות זהות בכל הרצה (שחזוריות)
- `n_init=10` – מריץ 10 אתחולים שונים ובוחר את הטוב ביותר

In [ ]:
# שלב 1: אימון המודל
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# שלב 2: הוספת עמודת Cluster ל-DataFrame
df_customers["Cluster"] = clusters

print(f"מודל KMeans אומן עם k={optimal_k}")
print(f"כל לקוח קיבל תווית קלאסטר (0 עד {optimal_k - 1})")

In [ ]:
# שלב 3: גודל כל קלאסטר
cluster_sizes = df_customers["Cluster"].value_counts().sort_index()
print("גודל כל קלאסטר:")
for cluster_id, size in cluster_sizes.items():
    pct = size / len(df_customers) * 100
    print(f"  Cluster {cluster_id}: {size} לקוחות ({pct:.1f}%)")

In [ ]:
# ממוצעים לכל קלאסטר (תכונות עיקריות)
profile_cols = ["BALANCE", "PURCHASES", "ONEOFF_PURCHASES", "INSTALLMENTS_PURCHASES",
                "CASH_ADVANCE", "CASH_ADVANCE_TRX", "PURCHASES_TRX",
                "CREDIT_LIMIT", "PAYMENTS", "MINIMUM_PAYMENTS", "PRC_FULL_PAYMENT",
                "PURCHASES_FREQUENCY"]
cluster_profiles = df_customers.groupby("Cluster")[profile_cols].mean()
cluster_profiles.round(2)

### פרשנות הקלאסטרים

לאחר שה-KMeans חילק את הלקוחות לקבוצות, התפקיד שלנו כאנליסטים הוא לתת משמעות עסקית לכל קבוצה.

In [ ]:
# פרשנות אוטומטית של הקלאסטרים
cluster_names = {}

for c in range(optimal_k):
    profile = cluster_profiles.loc[c]
    purch = profile["PURCHASES"]
    cash = profile["CASH_ADVANCE"]
    freq = profile["PURCHASES_FREQUENCY"]
    overall_avg_purch = cluster_profiles["PURCHASES"].mean()
    overall_avg_cash = cluster_profiles["CASH_ADVANCE"].mean()

    if cash > purch and cash > overall_avg_cash:
        name = "Cash-Advance Seekers"
    elif purch > overall_avg_purch * 2 and freq > 0.7:
        name = "High-Value Customers"
    elif freq > 0.5 and purch > overall_avg_purch * 0.5:
        name = "Frequent Buyers"
    else:
        name = "Low-Activity Customers"

    cluster_names[c] = name
    size = cluster_sizes[c]

    print(f"\nCluster {c} \u2013 \"{name}\" ({size} לקוחות)")
    print(f"  BALANCE: {profile['BALANCE']:,.0f} | PURCHASES: {purch:,.0f} | CASH_ADVANCE: {cash:,.0f}")
    print(f"  CREDIT_LIMIT: {profile['CREDIT_LIMIT']:,.0f} | PAYMENTS: {profile['PAYMENTS']:,.0f}")
    print(f"  תדירות רכישות: {freq:.2f} | אחוז תשלום מלא: {profile['PRC_FULL_PAYMENT']:.2f}")

### ויזואליזציה - דשבורד קלאסטרים

In [ ]:
fig_clust, axes_cl = plt.subplots(2, 3, figsize=(20, 12))
fig_clust.suptitle("K-Means Clustering \u2013 Customer Segmentation Dashboard",
                   fontsize=18, fontweight="bold", y=1.02)

cluster_colors = ["#3498db", "#e74c3c", "#2ecc71", "#9b59b6", "#f39c12"]

# גרף 1: גודל כל קלאסטר
bars = axes_cl[0, 0].bar(
    [f"C{i}\n{cluster_names.get(i, '')}" for i in range(optimal_k)],
    cluster_sizes.values,
    color=cluster_colors[:optimal_k], edgecolor="white"
)
for bar, val in zip(bars, cluster_sizes.values):
    axes_cl[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
                       str(val), ha="center", fontweight="bold", fontsize=10)
axes_cl[0, 0].set_title("Cluster Sizes", fontsize=13, fontweight="bold")
axes_cl[0, 0].set_ylabel("Number of Customers")

# גרף 2: ממוצע BALANCE ו-PURCHASES לכל קלאסטר
x_pos = np.arange(optimal_k)
width = 0.35
axes_cl[0, 1].bar(x_pos - width/2, cluster_profiles["BALANCE"], width,
                  label="Balance", color="#3498db", alpha=0.8)
axes_cl[0, 1].bar(x_pos + width/2, cluster_profiles["PURCHASES"], width,
                  label="Purchases", color="#e74c3c", alpha=0.8)
axes_cl[0, 1].set_title("Avg Balance vs Purchases by Cluster", fontsize=13, fontweight="bold")
axes_cl[0, 1].set_xlabel("Cluster")
axes_cl[0, 1].set_ylabel("Amount")
axes_cl[0, 1].set_xticks(x_pos)
axes_cl[0, 1].set_xticklabels([f"C{i}" for i in range(optimal_k)])
axes_cl[0, 1].legend()

# גרף 3: ממוצע CASH_ADVANCE לכל קלאסטר
axes_cl[0, 2].bar([f"C{i}" for i in range(optimal_k)],
                  cluster_profiles["CASH_ADVANCE"],
                  color=cluster_colors[:optimal_k], edgecolor="white")
axes_cl[0, 2].set_title("Avg Cash Advance by Cluster", fontsize=13, fontweight="bold")
axes_cl[0, 2].set_ylabel("Cash Advance Amount")

# גרף 4: Scatter PURCHASES vs BALANCE צבוע לפי קלאסטר
for c in range(optimal_k):
    mask = df_customers["Cluster"] == c
    axes_cl[1, 0].scatter(df_customers.loc[mask, "PURCHASES"],
                          df_customers.loc[mask, "BALANCE"],
                          alpha=0.4, color=cluster_colors[c],
                          label=f"C{c}: {cluster_names.get(c, '')}",
                          s=15, edgecolor="white", linewidth=0.3)
axes_cl[1, 0].set_title("Purchases vs Balance by Cluster", fontsize=13, fontweight="bold")
axes_cl[1, 0].set_xlabel("Purchases")
axes_cl[1, 0].set_ylabel("Balance")
axes_cl[1, 0].legend(fontsize=8)

# גרף 5: ממוצע CREDIT_LIMIT ו-PAYMENTS לכל קלאסטר
axes_cl[1, 1].bar(x_pos - width/2, cluster_profiles["CREDIT_LIMIT"], width,
                  label="Credit Limit", color="#2ecc71", alpha=0.8)
axes_cl[1, 1].bar(x_pos + width/2, cluster_profiles["PAYMENTS"], width,
                  label="Payments", color="#9b59b6", alpha=0.8)
axes_cl[1, 1].set_title("Avg Credit Limit vs Payments by Cluster", fontsize=13, fontweight="bold")
axes_cl[1, 1].set_xlabel("Cluster")
axes_cl[1, 1].set_ylabel("Amount")
axes_cl[1, 1].set_xticks(x_pos)
axes_cl[1, 1].set_xticklabels([f"C{i}" for i in range(optimal_k)])
axes_cl[1, 1].legend()

# גרף 6: תדירויות ואחוזים ממוצעים לכל קלאסטר
freq_cols = ["PURCHASES_FREQUENCY", "ONEOFF_PURCHASES_FREQUENCY",
             "PURCHASES_INSTALLMENTS_FREQUENCY", "CASH_ADVANCE_FREQUENCY", "PRC_FULL_PAYMENT"]
freq_means = df_customers.groupby("Cluster")[freq_cols].mean()
freq_means.T.plot(kind="bar", ax=axes_cl[1, 2], color=cluster_colors[:optimal_k],
                  edgecolor="white", alpha=0.85)
axes_cl[1, 2].set_title("Avg Frequencies & Ratios by Cluster", fontsize=13, fontweight="bold")
axes_cl[1, 2].set_ylabel("Average Value (0-1)")
axes_cl[1, 2].legend([f"C{i}" for i in range(optimal_k)], fontsize=8)
axes_cl[1, 2].tick_params(axis="x", rotation=25)

plt.tight_layout()
plt.savefig("clustering_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()

---
## חלק ו' - סיכום ומסקנות עסקיות

### 1. תובנות עסקיות מהתפלגות הקלאסטרים

| סגמנט | % | תובנה |
|--------|---|--------|
| **Frequent Buyers** | 37.4% | לקוחות נאמנים - חשוב לשמר אותם |
| **Low-Activity** | 44.6% | הפוטנציאל הגדול ביותר לצמיחה |
| **Cash-Advance Seekers** | 13.6% | סיכון אשראי - דורשים מעקב |
| **High-Value** | 4.5% | פרימיום, רווחיים ביותר |

**תובנה מרכזית:** לקוחות כרטיסי אשראי **אינם קבוצה הומוגנית**. יש הבדלים מהותיים בדפוסי השימוש.

### 2. המלצות לצוות השיווק

In [ ]:
recommendations = {
    "Cash-Advance Seekers": (
        "שיווק: הצעת הלוואות אישיות בריבית נמוכה יותר כחלופה למשיכות מזומן.\n"
        "סיכון: מעקב צמוד \u2013 משיכות מזומן תכופות מצביעות על בעיות תזרים.\n"
        "מבצע: בונוס על מעבר לרכישות רגילות במקום משיכות מזומן."
    ),
    "Frequent Buyers": (
        "שיווק: תוכנית נאמנות עם נקודות/cashback על רכישות.\n"
        "סיכון: נמוך \u2013 לקוחות יציבים שמשלמים בזמן.\n"
        "מבצע: הנחות בחנויות פופולריות, שדרוג כרטיס עם הטבות."
    ),
    "High-Value Customers": (
        "שיווק: שירות VIP, כרטיס פרימיום עם הטבות בלעדיות.\n"
        "סיכון: נמוך \u2013 אבל חשוב לשמור על שביעות רצון כדי שלא יעזבו.\n"
        "מבצע: הזמנות לאירועים, גישה ללאונג'ים בשדות תעופה, ביטוח נסיעות."
    ),
    "Low-Activity Customers": (
        "שיווק: קמפיין הפעלה \u2013 'חזור להשתמש בכרטיס וקבל X'.\n"
        "סיכון: סיכון לנטישה (churn) \u2013 צריך לפעול לפני שעוזבים.\n"
        "מבצע: 0% עמלה על 3 חודשים ראשונים, cashback מוגבר."
    ),
}

for c in range(optimal_k):
    name = cluster_names[c]
    print(f"\nCluster {c} \u2013 \"{name}\":")
    if name in recommendations:
        print(recommendations[name])

### סיכום

מודל K-Means מאפשר לחברת CardWise לעבור **משיווק גורף לשיווק ממוקד** (Targeted Marketing), מה שצפוי לשפר את שימור הלקוחות, להגדיל הכנסות, ולנהל סיכוני אשראי בצורה חכמה יותר.